# Prework
1. Create an account on SynBioHub
2. Download the data file they will need to attach
3. Download the SBOL document with the second part

# Installation
## Python
`pip install pysbol`

# Get Device from Synbiohub
In this section, we are going to download a device from SynBioHub. This device will contain a number of components, sequences, and other objects as well.

In [ ]:
import sbol

# Start an interface to the part shop
part_shop = sbol.PartShop("https://synbiohub.org")


In [ ]:
# Search for records from the interlab study
records = part_shop.search("interlab")
for record in records:
    print("{}: {}".format(record.displayId, record))

In [ ]:
# Read the strong device into a new document
doc = sbol.Document()
strong_comp_uri = records[3].identity
part_shop.pull(strong_comp_uri, doc)


In [ ]:
# Explore the strong device document
for obj in doc:
    print("{}: {}".format(sbol.parseClassName(obj.type), obj))

# Extracting a ComponentDefinition from a Pre-existing Device
In this section, we will read in a new device from an SBOL compliant XML. The goal here is to extract the promoter from the device

# Creating a New Device
In this section, we will create a new device by swapping in the promoter from the XML into the device from the iterlab study

# Adding Data to a Device via an Attachement
Now that we have a new device, we will add data to it as an attachment

# Uploading the Device back to SynBioHub
Finally, we can create a new collection on SynBioHub with the new device and its attachement.